<a href="https://colab.research.google.com/github/22022658NguyenTienKhoi/Pretraining-data/blob/main/pretraining_vietnamese_data_curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Processing High-Quality Vietnamese Data: Viettel’s Success with NVIDIA NeMo Curator

Open-source [large language models (LLMs)](https://www.nvidia.com/en-us/glossary/large-language-models/) excel in English but struggle with other languages, especially in Southeast Asia. This is primarily due to a lack of training data in these languages, limited understanding of local cultures, and insufficient tokens to capture unique linguistic structures and expressions. To fully meet customer needs, enterprises in non-English-speaking countries must go beyond generic models and customize them to capture the nuances of their local languages, ensuring a seamless and impactful customer experience.

In this tutorial, we will use NeMo Curator to process high-quality [Vietnamese data](https://huggingface.co/datasets/VTSNLP/vietnamese_curated_dataset). We will guide you through the data curation pipeline used and share sample code for each stage.

## Table of Contents
- **1. [Prerequisites and Environment setups](#prerequisites-and-environment-setups)**
- **2. [Data Collecting](#data-collecting)**
- **3. [Data Curation flow](#data-curation-flow)**
    - a. [Unicode reformatting](#unicode-reformatting)
    - b. [Adding Custom IDs to Documents](#adding-custom-ids-to-documents)
    - c. [Exact deduplication](#exact-deduplication)
    - d. [Heuristic Quality Filtering](#heuristic-quality-filtering)
    - e. [Classifier-based Quality Filtering](#classifier-based-quality-filtering)

## Prerequisites and Environment setups

In [ ]:
!git clone https://github.com/NVIDIA/NeMo-Curator.git
!pip install --extra-index-url https://pypi.nvidia.com "./NeMo-Curator[all]"

Cloning into 'NeMo-Curator'...
remote: Enumerating objects: 2804, done.
remote: Counting objects: 100% (1116/1116), done.
remote: Compressing objects: 100% (537/537), done.
remote: Total 2804 (delta 765), reused 729 (delta 578), pack-reused 1688 (from 1)
Receiving objects: 100% (2804/2804), 5.54 MiB | 24.37 MiB/s, done.
Resolving deltas: 100% (1719/1719), done.
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Processing ./NeMo-Curator
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 92.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Install NeMo Curator by following the instructions to install the CPU and CUDA-accelerated modules in the README file of the [NeMo Curator repository](https://github.com/NVIDIA/NeMo-Curator/tree/main).

Next, install these additional packages:

In [ ]:
!pip install datasets
!pip install jsonlines

To proceed with data processing, we need to set up a Dask environment. Dask is a flexible, open-source library that enables parallel and distributed computing in Python, allowing us to scale computations across multiple cores or even clusters. By distributing tasks, Dask makes the data handling process significantly faster and more efficient.

**Note:** This notebook was run on a single DGX A100 GPU, with a 128-core CPU and 2TB of RAM to handle the dataset size. Depending on your dataset and computing resources, you may need to adjust the Dask worker configuration below accordingly.

In [ ]:
from dask.distributed import Client, LocalCluster

# Start a Dask cluster with 12 workers, each limited at 64GB of memory.
# You might need to adjust these numbers according to your computing resources.
cluster = LocalCluster(n_workers=12, processes=True, memory_limit="12GB")
client = Client(cluster)

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:42503
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42641'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39829'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37819'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42123'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44211'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45649'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39659'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40409'
INFO:distributed.nanny:        Start Na

## Data Collecting

Each dataset is accessed and downloaded using the Hugging Face Hub, with additional steps required for OSCAR (the Vietnamese subset dataset, version 23.01, an aggregation of web-crawled data) due to its access restrictions. For OSCAR, you need to accept the conditions on the [dataset page](https://huggingface.co/datasets/oscar-corpus/OSCAR-2301) and use a [Hugging Face access token](https://huggingface.co/docs/hub/en/security-tokens) for downloading.

**Download and Convert Datasets to Parquet**

The conversion of dataset into Parquet format facilitates efficient handling and processing of large datasets.

**Combine and Standardize Format**

We then combine them into a single dataset, keeping only the "text" column.

In [ ]:
from datasets import concatenate_datasets

# Combine datasets and standardize format
datasets = [os.path.join(data_dir, file) for file in ["wiki_vi_231101.parquet", "c4_vi.parquet", "oscar_vi.parquet", "binhvq_news_train.parquet"]]

data_files = {"train": datasets[0]}
ds = load_hf_dataset("parquet", data_files=data_files)
ds = ds["train"].remove_columns([col for col in ds["train"].column_names if col != "text"])

for d in datasets[1:]:
    ds_ = load_hf_dataset("parquet", data_files={"train": d})
    ds_ = ds_["train"].remove_columns([col for col in ds_["train"].column_names if col != "text"])
    ds = concatenate_datasets([ds, ds_])

Generating train split: 1288680 examples [00:03, 332051.80 examples/s]


**Shard the Combined Dataset**

The combined dataset is then sharded into smaller chunks. Sharding is performed to distribute the data evenly across multiple workers in the Dask cluster, facilitating efficient parallel processing during the data curation stages.

In [ ]:
# Define paths for raw data
raw_data_directory = os.path.join(data_dir, "raw")

# Shard the dataset
num_shards = 256
for shard_idx in range(num_shards):
    shard = ds.shard(index=shard_idx, num_shards=num_shards)
    shard.to_parquet(os.path.join(raw_data_directory, f"{shard_idx}.parquet"))

Creating parquet from Arrow format: 100%|██████████| 6/6 [00:00<00:00, 119.30ba/s]


## Data Curation flow

### Unicode reformatting

Unicode reformatting is an essential preprocessing step to ensure that text data is standardized and free of encoding errors, which are common in web-crawled datasets.

In [ ]:
raw_data_directory = '/content/raw'

In [ ]:
data_dir = '/content'

In [ ]:
from nemo_curator import Modify
from nemo_curator.modifiers import UnicodeReformatter
from nemo_curator.utils.distributed_utils import read_data, write_to_disk
from nemo_curator.utils.file_utils import get_all_files_paths_under
from nemo_curator.datasets import DocumentDataset
import os
# Define paths for Unicode formatted data
unicode_formatted_output_path = os.path.join("/content/", "formatted")

# Load the raw data
def load_dataset(input_data_dir, file_type="json"):
    files = list(get_all_files_paths_under(input_data_dir))
    raw_data = read_data(files, file_type=file_type, backend="pandas", add_filename=True)
    dataset = DocumentDataset(raw_data)

    return dataset

raw_data = load_dataset(raw_data_directory, file_type="json")
raw_data.df['content'] = raw_data.df['content'].astype(str)
# Initialize the Unicode reformatter
cleaner = Modify(UnicodeReformatter(),text_field="content")

# Apply Unicode reformatting
cleaned_data = cleaner(raw_data)

# Save the cleaned data to disk
write_to_disk(cleaned_data.df, unicode_formatted_output_path, write_to_filename=True, output_type="parquet")

Reading 6 files
Writing to disk complete for 6 partitions


### Adding Custom IDs to Documents

Before proceeding with further curation steps, it is advisable to preprocess the dataset by adding a unique ID to each document. These IDs serve as trackers that help in identifying duplicate or low-quality documents throughout the curation process, ensuring that each document remains uniquely identifiable throughout processing. <br>

NeMo Curator offers an `AddId` class, which allows users to insert custom IDs into documents using a specified prefix format, such as `<prefix>_<id>`.

In [ ]:
from pickle import load
from nemo_curator import AddId

# Define paths for input data and output with added IDs
add_id_input_data_dir = unicode_formatted_output_path
added_id_output_path = os.path.join(data_dir, "add_id")
add_ID_id_prefix = "VI_"

# Load the formatted dataset
dataset = load_dataset(add_id_input_data_dir, file_type="parquet")

# Initialize the AddId class with a specified prefix and start index
add_id = AddId(id_field="id", id_prefix=add_ID_id_prefix, start_index=0)

# Apply the ID addition to the dataset
id_dataset = add_id(dataset)

# Save the dataset with added IDs to disk
write_to_disk(id_dataset.df, output_file_dir=added_id_output_path, write_to_filename=True, output_type="parquet")

Reading 6 files
Writing to disk complete for 6 partitions


### Exact deduplication

Exact deduplication removes identical duplicates from the dataset. By eliminating exact duplicates, we ensure that each data point contributes uniquely to the training process, enhancing the diversity and overall quality of the dataset.

In this stage, we’ll leverage GPU acceleration by utilizing a Dask CUDA cluster. Since the current cluster is CPU-based, we need to shut it down and start a new one with GPU support.

To close the existing cluster:


In [ ]:
client.cluster.close()
client.shutdown()

Then, to initialize the GPU Dask cluster:

In [ ]:
from nemo_curator.utils.distributed_utils import get_client

def pre_imports():
    import cudf

client = get_client(cluster_type="gpu", set_torch_to_use_rmm=False)
client.run(pre_imports)

{'tcp://127.0.0.1:38425': None}

**Below is the implementation for exact deduplication:**

Imports and directory preparation:

In [ ]:
import os
from nemo_curator.modules import ExactDuplicates
from nemo_curator.datasets import DocumentDataset

# Define input and output paths
exact_dedup_input_dataset_dir = added_id_output_path
exact_dedup_base_output_path = os.path.join(data_dir, "exact_dedup")
exact_dedup_log_dir = os.path.join(exact_dedup_base_output_path, "log")
exact_dedup_output_dir = os.path.join(exact_dedup_base_output_path, "data")
deduped_output_dir = os.path.join(data_dir, "remove_duplicate")

# Create directories for logs and output
!mkdir -p {exact_dedup_log_dir}
!mkdir -p {exact_dedup_output_dir}
!mkdir -p {deduped_output_dir}

Set parameters and load dataset:

In [ ]:
# Parameters for ExactDuplicates
exact_dedup_dataset_id_field = "id"
exact_dedup_dataset_text_field = "content"

# Load the input dataset
input_dataset = DocumentDataset.read_parquet(exact_dedup_input_dataset_dir, backend="pandas")

Reading 6 files


Initialize and run deduplication:

In [ ]:
# Initialize and run exact deduplication
exact_dup = ExactDuplicates(
    logger=exact_dedup_log_dir,
    id_field=exact_dedup_dataset_id_field,
    text_field=exact_dedup_dataset_text_field,
    hash_method="md5",
    cache_dir=exact_dedup_output_dir
)
duplicates = exact_dup(dataset=input_dataset)

print(f"Number of exact duplicate files: {len(duplicates)}")

INFO:ExactDuplicates:Starting lazy hash generation
INFO:ExactDuplicates:Lazy hash generation complete for 6 partitions
INFO:ExactDuplicates:Starting execution for ExactDedup
INFO:ExactDuplicates:Time taken for Exact Dedup Computation = 2.0106115341186523s and output written at /content/exact_dedup/data/_exact_duplicates.parquet


Number of exact duplicate files: 16406


Remove duplicates and save final dataset:

In [ ]:
# Load the dataset and exact duplicates to identify and remove duplicate IDs
input_dataset = load_dataset(exact_dedup_input_dataset_dir, file_type="parquet")
exact_duplicates = DocumentDataset.read_parquet(
    os.path.join(exact_dedup_output_dir, "_exact_duplicates.parquet"), backend="pandas"
)

# Extract list of duplicate document IDs
exact_docs_to_remove = exact_duplicates.df.map_partitions(
    lambda x: x[x._hashes.duplicated(keep="first")]
)

# Remove duplicated documents from the input dataset
result = input_dataset.df[
    ~input_dataset.df[exact_dedup_dataset_id_field].isin(exact_docs_to_remove[exact_dedup_dataset_id_field].compute())
]

# Save the final deduplicated dataset
write_to_disk(result, output_file_dir=deduped_output_dir, write_to_filename=True, output_type="parquet")

Reading 6 files
Reading 1 files
Writing to disk complete for 6 partitions


Close the GPU Dask cluster:

In [ ]:
client.cluster.close()
client.shutdown()

### Heuristic Quality Filtering

Heuristic quality filtering is designed to enhance the quality of the dataset by removing low-quality content based on predefined heuristics. This approach involves applying a series of filters to the dataset to eliminate undesirable data characteristics such as excessive special characters, overly short or long texts, or other criteria that could negatively impact model performance.

We use a YAML file to define the heuristic filters. The configuration can be found [here](https://raw.githubusercontent.com/NVIDIA/NeMo-Curator/main/config/heuristic_filter_non-en.yaml). This file lists the filtering criteria and settings used to build a filter pipeline. You can customize the filters or change thresholds based on your needs. The `filter_pipeline` helper reads the YAML settings and applies each filter to the dataset step by step.\n

Recreate a CPU Dask cluster:

In [ ]:
# Start a Dask cluster with 12 workers, each limited at 64GB of memory.
# You might need to adjust these numbers according to your computing resources

cluster = LocalCluster(n_workers=12, processes=True, memory_limit="64GB")
client = Client(cluster)

In [ ]:
from nemo_curator.utils.config_utils import build_filter_pipeline
import warnings

# Define paths for input data and output data after heuristic filtering
HF_input_data_dir = deduped_output_dir
HF_output_path = os.path.join(data_dir, "heuristic_filtering")

# Create a directory for the configuration file if it doesn't exist
os.makedirs("config", exist_ok=True)
# Download the YAML configuration file for heuristic filtering
!wget https://raw.githubusercontent.com/NVIDIA/NeMo-Curator/main/config/heuristic_filter_non-en.yaml -O ./config/heuristic_filter_non-en.yaml

# Specify the path to the configuration file
filter_config_file = "./config/heuristic_filter_non-en.yaml"
os.makedirs(HF_output_path, exist_ok=True)

# Load the filters from the YAML configuration file


--2024-12-01 09:28:46--  https://raw.githubusercontent.com/NVIDIA/NeMo-Curator/main/config/heuristic_filter_non-en.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3911 (3.8K) [text/plain]
Saving to: ‘./config/heuristic_filter_non-en.yaml’

./config/heuristic_ 100%[===================>]   3.82K  --.-KB/s    in 0s      

2024-12-01 09:28:46 (52.7 MB/s) - ‘./config/heuristic_filter_non-en.yaml’ saved [3911/3911]



In [ ]:
import pandas as pd
df = pd.read_parquet('/content/remove_duplicate/education.parquet')
df = df.dropna(subset=['content'])

In [ ]:
df

,_class,_id,category,content,create_at,domain,id,title,url
0,com.news.scanner.entity.News,{'$oid': '66fd5cc447ad95729b9c4dbc'},"[lịch sử, đại học quốc gia, giáo dục]",LỊCH SỬ HÌNH THÀNH VÀ PHÁT TRIỂN Ngày 21/12/19...,{'$date': '2024-10-02T14:46:28.536Z'},https://education.vnu.edu.vn/,VI_-0000000000,Lịch sử - Trường Đại học giáo dục,https://education.vnu.edu.vn/index.php/WebCont...
1,com.news.scanner.entity.News,{'$oid': '66fd5cca47ad95729b9c4dbd'},"[đại học quốc gia, giáo dục, sứ mạng - tầm nhìn]","SỨ MẠNG, TẦM NHÌN VÀ GIÁ TRỊ CỐT LÕI Sứ mạng T...",{'$date': '2024-10-02T14:46:34.011Z'},https://education.vnu.edu.vn/,VI_-0000000001,Sứ mạng - Tầm nhìn - Trường Đại học giáo dục,https://education.vnu.edu.vn/index.php/WebCont...
2,com.news.scanner.entity.News,{'$oid': '66fd5cd047ad95729b9c4dbe'},"[đại học quốc gia, giáo dục]",Danh mục luận văn thạc sỹ Quản lý Giáo dục kho...,{'$date': '2024-10-02T14:46:40.431Z'},https://education.vnu.edu.vn/,VI_-0000000002,Tin tức - Trường Đại học giáo dục,https://education.vnu.edu.vn/index.php/WebCont...
3,com.news.scanner.entity.News,{'$oid': '66fd5e4a22d59a51c4518727'},"[video giới thiệu, đại học quốc gia, giáo dục]",Trường Đại học Giáo dục - Đại học Quốc gia Hà ...,{'$date': '2024-10-02T14:52:58.778Z'},https://education.vnu.edu.vn/,VI_-0000000003,Video giới thiệu - Trường Đại học giáo dục,https://education.vnu.edu.vn/index.php/WebCont...
4,com.news.scanner.entity.News,{'$oid': '66fd5e5722d59a51c4518728'},"[đại học quốc gia, giáo dục, văn bản hướng dẫn]",ĐÀO TẠO ĐẠI HỌC CHÍNH QUY 337/GDTC&TT-ĐT Hướng...,{'$date': '2024-10-02T14:53:11.362Z'},https://education.vnu.edu.vn/,VI_-0000000004,Văn bản hướng dẫn - Trường Đại học giáo dục,https://education.vnu.edu.vn/index.php/WebCont...
...,...,...,...,...,...,...,...,...,...
1860,com.news.scanner.entity.News,{'$oid': '66fd893d22d59a51c4518e68'},[điểm trúng tuyển vào trường đh giáo dục – cao...,Điểm trúng tuyển vào Trường ĐH Giáo dục – cao ...,{'$date': '2024-10-02T17:56:13.642Z'},https://education.vnu.edu.vn/,VI_-0000001860,Tin tức - Trường Đại học giáo dục,https://education.vnu.edu.vn/index.php/WebCont...
1892,com.news.scanner.entity.News,{'$oid': '66fd8a1422d59a51c4518e88'},"[đại học quốc gia, giáo dục, trường đhgd thông...","Trường ĐHGD thông báo điểm chuẩn, hướng dẫn xá...",{'$date': '2024-10-02T17:59:48.834Z'},https://education.vnu.edu.vn/,VI_-0000001892,Tin tức - Trường Đại học giáo dục,https://education.vnu.edu.vn/index.php/WebCont...
1893,com.news.scanner.entity.News,{'$oid': '66fd8a1a22d59a51c4518e89'},[trường đại học giáo dục thông báo tuyển sinh ...,Trường Đại học Giáo dục Thông báo tuyển sinh đ...,{'$date': '2024-10-02T17:59:54.294Z'},https://education.vnu.edu.vn/,VI_-0000001893,Tin tức - Trường Đại học giáo dục,https://education.vnu.edu.vn/index.php/WebCont...
1946,com.news.scanner.entity.News,{'$oid': '66fd8b7622d59a51c4518ebe'},[thông báo xác nhận nhập học và tiếp nhận thí ...,Thông báo xác nhận nhập học và tiếp nhận thí s...,{'$date': '2024-10-02T18:05:42.081Z'},https://education.vnu.edu.vn/,VI_-0000001946,Tin tức - Trường Đại học giáo dục,https://education.vnu.edu.vn/index.php/WebCont...


In [ ]:
filter_pipeline = build_filter_pipeline('/content/config/heuristic_filter_non-en.yaml')

# Load the dataset
dataset =load_dataset('/content/remove_duplicate', file_type="parquet")

# Suppress specific warnings during filtering
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    # Apply the heuristic filters to the dataset
    result_data = filter_pipeline(dataset)

    # Save the filtered dataset to disk
    result_data.to_parquet(HF_output_path, write_to_filename=True)

Reading 6 files


ZeroDivisionError: division by zero

### Classifier-based Quality Filtering

Classifier-based filtering uses a trained classifier model to sort content as high or low quality, offering a smarter and more flexible way to handle diverse datasets that simple rules might miss.

**Prepare Data for Training Classifier**

To train a quality classifier, we need representative samples of both high-quality and low-quality content. For high-quality data, we use articles from Wikipedia's Vietnamese edition, which are generally well-structured and reliable. The low-quality samples come from unfiltered crawled Vietnamese news corpus.

In [ ]:
import os
from datasets import load_dataset as load_hf_dataset

In [ ]:
data_dir = '/content'

In [ ]:
# Paths for high-quality and low-quality sample data
hq_samples_path = os.path.join(data_dir, "classifier_filtering/train_samples/hq")
lq_samples_path = os.path.join(data_dir, "classifier_filtering/train_samples/lq")

# Load and shard the high-quality dataset
ds = load_hf_dataset("wikimedia/wikipedia", "20231101.vi")
num_shards = 8
for shard_idx in range(num_shards):
    shard = ds["train"].shard(index=shard_idx, num_shards=num_shards)
    shard.to_parquet(os.path.join(hq_samples_path, f"{shard_idx}.parquet"))

# Load and shard the low-quality dataset
ds = load_hf_dataset("vietgpt/binhvq_news_vi",split="train[:100000]")
num_shards = 32
for shard_idx in range(num_shards):
    shard = ds.shard(index=shard_idx, num_shards=num_shards)
    shard.to_parquet(os.path.join(lq_samples_path, f"{shard_idx}.parquet"))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/291M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/71.0M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/50.9M [00:00<?, ?B/s]

...

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

**Training Classifier**

The classifier is trained using FastText, which offers an efficient and effective method for text classification.

In [ ]:
from nemo_curator import Modify
from nemo_curator.utils.distributed_utils import write_to_disk
from nemo_curator.datasets import DocumentDataset

INFO:distributed.core:Event loop was unresponsive in Nanny for 3.23s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.23s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.23s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.23s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.23s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts an

In [ ]:
from nemo_curator.modifiers import FastTextLabelModifier
import fasttext
import random

# Function to create labeled samples
def create_samples(data_path, label, num_samples):
    raw_dataset = DocumentDataset.read_parquet(data_path, backend="pandas")
    label_quality = Modify(FastTextLabelModifier(label))
    labeled_dataset = label_quality(raw_dataset)
    labeled_samples = labeled_dataset.df.sample(frac=num_samples / len(labeled_dataset.df))

    return labeled_samples["text"].compute().values.tolist()

# Prepare training data
low_quality_samples = create_samples('classifier_filtering/train_samples/hq', "__label__lq", 1000)
high_quality_samples = create_samples('classifier_filtering/train_samples/lq', "__label__hq", 1000)
train_samples = low_quality_samples + high_quality_samples
random.shuffle(train_samples)

# Save training data to a file
train_file = "./cf_model_fasttext.train"
with open(train_file, "w", encoding="utf-8") as f:
    for sample in train_samples:
        f.write(sample + "\n")

# Train the FastText classifier
model = fasttext.train_supervised(input=train_file, lr=0.01, dim=100, epoch=5, wordNgrams=2)
model_path = "./cf_model_fasttext_model.bin"
model.save_model(model_path)

Reading 8 files
Reading 32 files


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Classify and Filter the Dataset**

Once trained, the classifier is used to filter the dataset, categorizing documents into high and low quality based on the learned distinctions.

In [ ]:
def load_dataset(input_data_dir, file_type="json"):
    files = list(get_all_files_paths_under(input_data_dir))
    raw_data = read_data(files, file_type=file_type, backend="pandas", add_filename=True)
    dataset = DocumentDataset(raw_data)

    return dataset

In [ ]:
from nemo_curator.filters import FastTextQualityFilter
from nemo_curator import ScoreFilter
from nemo_curator.utils.distributed_utils import read_data, write_to_disk
from nemo_curator.utils.file_utils import get_all_files_paths_under
from nemo_curator.datasets import DocumentDataset

# Define paths and load the dataset
CF_input_data_dir = '/content/drive/MyDrive/heuristic_filtering'
CF_output_path = os.path.join(data_dir, "classifier_filtering/output")
target_dataset = load_dataset(CF_input_data_dir, file_type="parquet")

# Set up the filtering pipeline
filter_pipeline = ScoreFilter(FastTextQualityFilter(model_path), score_field="quality_score", score_type=float,text_field = 'content')
filtered_dataset = filter_pipeline(target_dataset)

# Save the filtered dataset
write_to_disk(filtered_dataset.df, output_file_dir=CF_output_path, write_to_filename=True, output_type="parquet")

Reading 5 files
Writing to disk complete for 5 partitions


Close the CPU Dask cluster:

In [ ]:
client.cluster.close()
client.shutdown()

We have completed the notebook! For other techniques such as Fuzzy Deduplication or PII redaction, you can go to [NeMo Curator example scripts](https://github.com/NVIDIA/NeMo-Curator/tree/main/examples).

In [ ]:
!cp -r /content/remove_duplicate /content/drive/My\ Drive/

In [ ]:
import pandas as pd
import os
folder_path = '/content/drive/MyDrive/remove_duplicate'
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]

# Read and concatenate all DataFrames
df_list = [pd.read_parquet(file) for file in all_files]
df = pd.concat(df_list, ignore_index=True)


In [ ]:
df

,_class,_id,category,content,create_at,domain,id,title,url
0,com.news.scanner.entity.News,{'$oid': '66fcf9c32023476fe68e0d54'},"[đại học quốc gia, nhà xuất bản]",Sách sắp xuất bản Xem tất cả Sắp xuất bản Chợ ...,{'$date': '2024-10-02T07:44:03.327Z'},https://press.vnu.edu.vn/,VI_-0000039271,NXB ĐHQGHN - Nhà xuất bản Đại học Quốc gia Hà ...,https://press.vnu.edu.vn/
1,com.news.scanner.entity.News,{'$oid': '66fcf9c62023476fe68e0d55'},"[đại học quốc gia, nhà xuất bản, liên hệ]",Tên đầy đủ: NHÀ XUẤT BẢN ĐẠI HỌC QUỐC GIA HÀ N...,{'$date': '2024-10-02T07:44:06.582Z'},https://press.vnu.edu.vn/,VI_-0000039272,NXB ĐHQGHN - Nhà xuất bản Đại học Quốc gia Hà ...,https://press.vnu.edu.vn/lien-he
2,com.news.scanner.entity.News,{'$oid': '66fcf9ca2023476fe68e0d56'},"[đại học quốc gia, nhà xuất bản]",Chia sẻ: Tâm lý học xuyên văn hóa 550.000 ₫ Bả...,{'$date': '2024-10-02T07:44:10.547Z'},https://press.vnu.edu.vn/,VI_-0000039273,NXB ĐHQGHN - Nhà xuất bản Đại học Quốc gia Hà ...,https://press.vnu.edu.vn/tam-ly-hoc-xuyen-van-...
3,com.news.scanner.entity.News,{'$oid': '66fcf9cd2023476fe68e0d57'},"[sứ mệnh tầm nhìn, đại học quốc gia, nhà xuất ...",Sứ Mệnh Của Nhà Xuất Bản ĐHQGHN: Cung cấp nhữn...,{'$date': '2024-10-02T07:44:13.345Z'},https://press.vnu.edu.vn/,VI_-0000039274,NXB ĐHQGHN - Nhà xuất bản Đại học Quốc gia Hà ...,https://press.vnu.edu.vn/su-menh-tam-nhin
4,com.news.scanner.entity.News,{'$oid': '66fcf9d02023476fe68e0d58'},"[đại học quốc gia, nhà xuất bản]",Chia sẻ: Sống một cuộc đời trọn vẹn FLASH SALE...,{'$date': '2024-10-02T07:44:16.543Z'},https://press.vnu.edu.vn/,VI_-0000039275,NXB ĐHQGHN - Nhà xuất bản Đại học Quốc gia Hà ...,https://press.vnu.edu.vn/song-mot-cuoc-doi-tro...
...,...,...,...,...,...,...,...,...,...
27978,com.news.scanner.entity.News,{'$oid': '66a51bcdedd1964876955ff9'},[Tuyển Sinh],Xét tuyển & nhập học Phương thức xét tuyển Hom...,{'$date': '2024-07-27T16:09:49.336Z'},uet.vnu.edu,VI_-0000027443,Tham khảo điểm trúng tuyển vào Trường Đại học ...,https://tuyensinh.uet.vnu.edu.vn/ban-nen-biet/...
27979,com.news.scanner.entity.News,{'$oid': '66a51be2edd1964876955ffa'},[Tuyển Sinh],Xét tuyển & nhập học Phương thức xét tuyển Hom...,{'$date': '2024-07-27T16:10:10.166Z'},uet.vnu.edu,VI_-0000027444,SINH VIÊN TRƯỜNG ĐẠI HỌC CÔNG NGHỆ CÓ MẶT TRON...,https://tuyensinh.uet.vnu.edu.vn/ban-nen-biet/...
27980,com.news.scanner.entity.News,{'$oid': '66a51bf6edd1964876955ffb'},[Tuyển Sinh],Xét tuyển & nhập học Phương thức xét tuyển Hom...,{'$date': '2024-07-27T16:10:30.710Z'},uet.vnu.edu,VI_-0000027445,02 nhóm sinh viên Trường ĐHCN đạt giải thưởng ...,https://tuyensinh.uet.vnu.edu.vn/ban-nen-biet/...
27981,com.news.scanner.entity.News,{'$oid': '66a51bf7edd1964876955ffc'},[Tuyển Sinh],Xét tuyển & nhập học Phương thức xét tuyển Hom...,{'$date': '2024-07-27T16:10:31.603Z'},uet.vnu.edu,VI_-0000027446,"Sinh viên Trường Đại học Công nghệ, ĐHQGHN già...",https://tuyensinh.uet.vnu.edu.vn/ban-nen-biet/...


hf :6445  
dedup:27983  
dup :16406

In [ ]:
print(16406 + 27983)

44389


In [ ]:
print(16406/50834)

0.3227367509934296


In [ ]:
print((27983 - 6445)/50834)

0.4236928040287996


In [ ]:
print(6445/50834)

0.12678522248888538


In [ ]:
pip uninstall -y tensorflow && pip install tensorflow-cpu

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.0/230.0 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.3/381.3 kB 26.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: ke

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

# Load the model and tokenizer
config = AutoConfig.from_pretrained("VTSNLP/BERT-domain-classifier")
tokenizer = AutoTokenizer.from_pretrained("VTSNLP/BERT-domain-classifier")
model = AutoModelForSequenceClassification.from_pretrained("VTSNLP/BERT-domain-classifier").to('cuda')


In [ ]:
from tqdm import tqdm
def predict_domain_batch(texts, batch_size=32):
    all_predictions = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding="longest", max_length=128, truncation=True).to('cuda')
        outputs = model(**inputs).logits
        predicted_classes = torch.argmax(outputs, dim=1)
        all_predictions.extend([config.id2label[class_idx.item()] for class_idx in predicted_classes])
    return all_predictions


In [ ]:
print(predict_domain_batch('mot hai ba'))

['LABEL_5']


In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
df['predicted_domain'] = predict_domain_batch(df['content'].tolist(), batch_size=16)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 13.06 MiB is free. Process 2435 has 14.73 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 42.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
df.to_parquet('/content/predicted_domain.parquet')